In [1]:
#Import external functions
from image_processing import *
from video_processing import *
from utils import *
from custom_models import *
import pickle
import random
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint

Using TensorFlow backend.


In [2]:
# Load images for training
data_type = 'frames'
weight_file_path = 'saved_models/caffenet_single_rgb.hdf5'

classes_dictionary = pickle.load(open('data_a/action_dictionary.p','rb'))

In [3]:
# Single frame model definition
input_shape = (crop_height,crop_width,3)
seq_input = (seq_len,crop_height,crop_width,3)

#SGD Optimizer Parameters
base_lr = 0.001
momentum = 0.9

#Step-based LearningRate Parameters
epochs = 420
gamma = 0.1
stepsize = 210

In [4]:
#Create CNN-LSTM model
lstm_model = LSTMCaffeDonahueFunctional(seq_input=seq_input, num_labels=num_labels)

#Load Single Frame weights for faster training
lstm_model = load_single_frame_weights(lstm_model, weight_file_path)

# Configure the model for training
lstm_model = model_compile(model=lstm_model, base_lr=base_lr, momentum=momentum)

In [5]:
file = open('data_a/ucf101_split1_trainVideos.txt','r')
train_videos = file.readlines()
file.close()
N = len(train_videos)
random.shuffle(train_videos)
val_videos = train_videos[:N//3]
train_videos = train_videos[N//3:]

batch_size_train = 17
batch_size_val = 17

In [6]:
#### STEP-BASED DECAY
# To Keras ston SGD apo mono tou kanei time-based decay
# Protimoume Step decay, giati emfanizei kalutero convergence
# Isws 8a mporousa na dokimasw kai Adaptive Learning
#
# Possible solution : https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
# Possible solution : https://machinelearningmastery.com/using-learning-rate-schedules-deep-learning-models-python-keras/
####
def step_decay(epoch):
    initial_lrate = base_lr
    drop = gamma
    epochs_drop = stepsize
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))

In [7]:
for pseudoseed in range(iterations):
    train_generator = frame_generator(train_videos,batch_size_train,data_type,pseudoseed)
    val_generator = frame_generator(val_videos,batch_size_val,data_type,pseudoseed)

    filepath = 'saved_models/lstm_rgb_512_pseudo'+str(pseudoseed)+'.hdf5'
    checkpointer = ModelCheckpoint(filepath=filepath,verbose=1,save_best_only=True)
    loss_history = LossHistory()
    lrate = LearningRateScheduler(step_decay,verbose=1)

    history = lstm_model.fit_generator(generator=train_generator,
                                       steps_per_epoch=15,
                                       validation_data=val_generator,
                                       validation_steps=5,
                                       epochs=epochs,
                                       callbacks=[lrate,loss_history,checkpointer])

Epoch 1/420

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 35s 2s/step - loss: 5.3158 - acc: 0.0549 - top_k_categorical_accuracy: 0.1216 - val_loss: 5.3049 - val_acc: 0.0118 - val_top_k_categorical_accuracy: 0.1412

Epoch 00001: val_loss improved from inf to 5.30492, saving model to saved_models/lstm_rgb_512_pseudo0.hdf5
Epoch 2/420

Epoch 00002: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 25s 2s/step - loss: 5.2792 - acc: 0.0549 - top_k_categorical_accuracy: 0.1882 - val_loss: 5.1776 - val_acc: 0.1412 - val_top_k_categorical_accuracy: 0.3529

Epoch 00002: val_loss improved from 5.30492 to 5.17758, saving model to saved_models/lstm_rgb_512_pseudo0.hdf5
Epoch 3/420

Epoch 00003: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 25s 2s/step - loss: 5.2047 - acc: 0.1294 - top_k_categorical_accuracy: 0.2941 - val_loss: 5.1024 - val_ac


Epoch 00022: val_loss did not improve from 1.62764
Epoch 23/420

Epoch 00023: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.7531 - acc: 0.7725 - top_k_categorical_accuracy: 0.9216 - val_loss: 1.9306 - val_acc: 0.6588 - val_top_k_categorical_accuracy: 0.9059

Epoch 00023: val_loss did not improve from 1.62764
Epoch 24/420

Epoch 00024: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.6792 - acc: 0.8039 - top_k_categorical_accuracy: 0.9373 - val_loss: 1.5772 - val_acc: 0.8000 - val_top_k_categorical_accuracy: 0.9529

Epoch 00024: val_loss improved from 1.62764 to 1.57722, saving model to saved_models/lstm_rgb_512_pseudo0.hdf5
Epoch 25/420

Epoch 00025: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.6643 - acc: 0.7843 - top_k_categorical_accuracy: 0.9294 - val_loss: 1.5845 - val_acc

15/15 [==============================] - 24s 2s/step - loss: 1.0482 - acc: 0.9333 - top_k_categorical_accuracy: 0.9804 - val_loss: 1.1114 - val_acc: 0.8941 - val_top_k_categorical_accuracy: 0.9882

Epoch 00071: val_loss did not improve from 0.95618
Epoch 72/420

Epoch 00072: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.9666 - acc: 0.9686 - top_k_categorical_accuracy: 0.9882 - val_loss: 1.2054 - val_acc: 0.8588 - val_top_k_categorical_accuracy: 0.9765

Epoch 00072: val_loss did not improve from 0.95618
Epoch 73/420

Epoch 00073: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 1.0005 - acc: 0.9608 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.3066 - val_acc: 0.8353 - val_top_k_categorical_accuracy: 0.9765

Epoch 00073: val_loss did not improve from 0.95618
Epoch 74/420

Epoch 00074: LearningRateScheduler setting learning rate to 0.001.
15/15 [===

15/15 [==============================] - 24s 2s/step - loss: 0.9358 - acc: 0.9686 - top_k_categorical_accuracy: 0.9922 - val_loss: 1.1884 - val_acc: 0.9294 - val_top_k_categorical_accuracy: 0.9765

Epoch 00096: val_loss did not improve from 0.95618
Epoch 97/420

Epoch 00097: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.8906 - acc: 0.9804 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.9850 - val_acc: 0.9765 - val_top_k_categorical_accuracy: 0.9882

Epoch 00097: val_loss did not improve from 0.95618
Epoch 98/420

Epoch 00098: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.9083 - acc: 0.9725 - top_k_categorical_accuracy: 0.9961 - val_loss: 1.0315 - val_acc: 0.9176 - val_top_k_categorical_accuracy: 0.9882

Epoch 00098: val_loss did not improve from 0.95618
Epoch 99/420

Epoch 00099: LearningRateScheduler setting learning rate to 0.001.
15/15 [===

15/15 [==============================] - 24s 2s/step - loss: 0.9029 - acc: 0.9608 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.1321 - val_acc: 0.9059 - val_top_k_categorical_accuracy: 0.9765

Epoch 00121: val_loss did not improve from 0.95618
Epoch 122/420

Epoch 00122: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.8810 - acc: 0.9765 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.9582 - val_acc: 0.9294 - val_top_k_categorical_accuracy: 1.0000

Epoch 00122: val_loss did not improve from 0.95618
Epoch 123/420

Epoch 00123: LearningRateScheduler setting learning rate to 0.001.
15/15 [==============================] - 24s 2s/step - loss: 0.8894 - acc: 0.9843 - top_k_categorical_accuracy: 1.0000 - val_loss: 1.2888 - val_acc: 0.8824 - val_top_k_categorical_accuracy: 0.9529

Epoch 00123: val_loss did not improve from 0.95618
Epoch 124/420

Epoch 00124: LearningRateScheduler setting learning rate to 0.001.
15/15 [

...

15/15 [==============================] - 24s 2s/step - loss: 0.4741 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.5966 - val_acc: 0.9647 - val_top_k_categorical_accuracy: 1.0000

Epoch 00320: val_loss did not improve from 0.48976
Epoch 321/420

Epoch 00321: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 24s 2s/step - loss: 0.4746 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.7179 - val_acc: 0.9294 - val_top_k_categorical_accuracy: 0.9882

Epoch 00321: val_loss did not improve from 0.48976
Epoch 322/420

Epoch 00322: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 24s 2s/step - loss: 0.4756 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.6278 - val_acc: 0.9412 - val_top_k_categorical_accuracy: 1.0000

Epoch 00322: val_loss did not improve from 0.48976
Epoch 323/420

Epoch 00323: LearningRateScheduler setting learning rate to 0.0001.
15/1

15/15 [==============================] - 24s 2s/step - loss: 0.4736 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.7668 - val_acc: 0.9529 - val_top_k_categorical_accuracy: 0.9765

Epoch 00345: val_loss did not improve from 0.48976
Epoch 346/420

Epoch 00346: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 24s 2s/step - loss: 0.4736 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.7665 - val_acc: 0.9176 - val_top_k_categorical_accuracy: 0.9765

Epoch 00346: val_loss did not improve from 0.48976
Epoch 347/420

Epoch 00347: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 24s 2s/step - loss: 0.4749 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.6088 - val_acc: 0.9882 - val_top_k_categorical_accuracy: 0.9882

Epoch 00347: val_loss did not improve from 0.48976
Epoch 348/420

Epoch 00348: LearningRateScheduler setting learning rate to 0.0001.
15/1

15/15 [==============================] - 24s 2s/step - loss: 0.4733 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.6350 - val_acc: 0.9647 - val_top_k_categorical_accuracy: 0.9765

Epoch 00370: val_loss did not improve from 0.48976
Epoch 371/420

Epoch 00371: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 24s 2s/step - loss: 0.4734 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.5998 - val_acc: 0.9529 - val_top_k_categorical_accuracy: 1.0000

Epoch 00371: val_loss did not improve from 0.48976
Epoch 372/420

Epoch 00372: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 24s 2s/step - loss: 0.4744 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.6225 - val_acc: 0.9765 - val_top_k_categorical_accuracy: 0.9882

Epoch 00372: val_loss did not improve from 0.48976
Epoch 373/420

Epoch 00373: LearningRateScheduler setting learning rate to 0.0001.
15/1

15/15 [==============================] - 24s 2s/step - loss: 0.4740 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.5836 - val_acc: 0.9765 - val_top_k_categorical_accuracy: 0.9882

Epoch 00395: val_loss did not improve from 0.48976
Epoch 396/420

Epoch 00396: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 24s 2s/step - loss: 0.4727 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.7407 - val_acc: 0.9412 - val_top_k_categorical_accuracy: 0.9765

Epoch 00396: val_loss did not improve from 0.48976
Epoch 397/420

Epoch 00397: LearningRateScheduler setting learning rate to 0.0001.
15/15 [==============================] - 24s 2s/step - loss: 0.4726 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.7198 - val_acc: 0.9529 - val_top_k_categorical_accuracy: 0.9882

Epoch 00397: val_loss did not improve from 0.48976
Epoch 398/420

Epoch 00398: LearningRateScheduler setting learning rate to 0.0001.
15/1

15/15 [==============================] - 24s 2s/step - loss: 0.4736 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.6310 - val_acc: 0.9647 - val_top_k_categorical_accuracy: 1.0000

Epoch 00420: val_loss did not improve from 0.48976
